# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [2]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Create a model
The LUME-services Model provides an API to all model services and facilitates all model operations.

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu",
    description = "disgen-impact-cu-inj"
)
model

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.author = :author_1 AND model.laboratory = :laboratory_1 AND model.facility = :facility_1 AND model.beampath = :beampath_1 AND model.description = :description_1
INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2022, 10, 21, 4, 24, 20),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='disgen-impact-cu-inj'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [5]:
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

In [6]:
### NOTE: The below cell will raise an error if run 2x

In [7]:
# create a project
project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)
scheduling_service.create_project("test")

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


You can now find this project in you Prefect UI at http://localhost:8080


![project](https://slaclab.github.io/lume-services/files/project_nav.png)

## Create a deployment for your model
Replace `source_path` with the path to your release tarball below:

In [8]:
source_path = "https://github.com/slaclab/lume-distgen-impact-cu-inj/releases/download/v0.1/lume_distgen_impact_cu_inj-0.1.tar.gz"

# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.models.model:installing package
INFO:lume_services.environment.solver:https://github.com/slaclab/lume-distgen-impact-cu-inj/releases/download/v0.1/lume_distgen_impact_cu_inj-0.1.tar.gz saved to /var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/tmpsjzsmp85/lume_distgen_impact_cu_inj-0.1.tar.gz
INFO:lume_services.environment.solver:Uninstall complete
INFO:lume_services.environment.solver:Installing dependencies...
INFO:lume_services.environment.solver:Dependency installation complete
INFO:lume_services.environment.solver:https://github.com/slaclab/lume-distgen-impact-cu-inj/releases/download/v0.1/lume_distgen_impact_cu_inj-0.1.tar.gz saved to /var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/tmp9ud8w_wh/lume_distgen_impact_cu_inj-0.1.tar.gz


['/Users/jacquelinegarrahan/miniconda3/envs/lume-distgen-impact-cu-inj-dev/bin/python', '-m', 'pip', 'install', '--no-deps', '-v', '/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/tmp9ud8w_wh/lume_distgen_impact_cu_inj-0.1.tar.gz']


  Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-751lg_ad/lume_distgen_impact_cu_inj.egg-info
  writing /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-751lg_ad/lume_distgen_impact_cu_inj.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-751lg_ad/lume_distgen_impact_cu_inj.egg-info/dependency_links.txt
  writing entry points to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-751lg_ad/lume_distgen_impact_cu_inj.egg-info/entry_points.txt
  writing requirements to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-751lg_ad/lume_distgen_impact_cu_inj.egg-info/requires.txt
  writing top-level names to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-751lg_ad/lume_distgen_impact_cu_inj.egg-info/top_level.txt
  writing manifest

INFO:lume_services.environment.solver:Dependency installation complete
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 AND deployment.version = :version_1
INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO deployment (version, package_import_name, asset_dir, source, sha256, image, is_live, model_id) VALUES (:version, :package_import_name, :asset_dir, :source, :sha256, :image, :is_live, :model_id)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO deployment (version, package_import_name, asset_dir, source, sha256, image, is_live, model_id) VALUES (:version, :package_import_name, :asset_dir, :source, :sha256, :image, :is_live, :model_id)
INFO:lume_ser

Flow URL: http://localhost:8080/default/flow/4b026886-d91e-43da-86c1-1a431cb924c3
 └── ID: 2f97c230-c8fe-4ada-93fd-fca51a2ce2c0
 └── Project: test
 └── Labels: ['lume-services']


## Run the Prefect workflow directly

In [9]:
# set up the parameters
import os
from lume_distgen_impact_cu_inj import DEFAULT_VCC_ARRAY
from datetime import datetime

# working directory
dir_path = os.getcwd()

# lets make directories for the archive file and the dashboard images
try:
    os.mkdir(f"{dir_path}/archive")
except FileExistsError:
    pass # already exists

try:
    os.mkdir(f"{dir_path}/dashboard")
except FileExistsError:
    pass # already exists

archive_dir = f"{dir_path}/archive"
dashboard_dir = f"{dir_path}/dashboard"

distgen_inputs = {
    "vcc_resolution" : 9,
    "vcc_resolution_units" : "um",
    "vcc_size_y" : 480,
    "vcc_size_x": 640,
    "vcc_array": DEFAULT_VCC_ARRAY.tolist(), # neet to convert to json serializable input for passed data
    "total_charge": 1.51614e+09
}

distgen_configuration = {}
distgen_settings = {
    'n_particle': 10000,
    "t_dist:length:value":  4 * 1.65   #  Inferred pulse stacker FWHM: 4 ps, converted to tukey length
}


pv_collection_isotime = datetime.now()

# in this case, will be using conda installation of impact
command="ImpactTexe"
command_mpi=""
use_mpi=False
mpi_run="mpirun -n {nproc} --use-hwthread-cpus {command_mpi}"

impact_configuration = {
    "command": command,
    "command_mpi": command_mpi,
    "use_mpi": use_mpi,
    "workdir": str(dir_path),
    "mpi_run": mpi_run
}

impact_settings = {
    "header:Nx": 32,
    "header:Ny": 32,
    "header:Nz": 32,
    "stop": 16.5,
    #  "stop": 8,
    "numprocs": 1,
    "timeout": 1000,
    "total_charge": 0, # for debugging
}


impact_inputs = {"SOL1:solenoid_field_scale": 0.47235,
                "CQ01:b1_gradient":  -0.00133705,
                "SQ01:b1_gradient": 0.000769202,
                "L0A_phase:dtheta0_deg": 0,
                "L0B_phase:dtheta0_deg": -2.5,
                "L0A_scale:voltage": 58,
                "L0B_scale:voltage": 69.9586,
                "QA01:b1_gradient": -3.25386,
                "QA02:b1_gradient": 2.5843,
                "QE01:b1_gradient": -1.54514,
                "QE02:b1_gradient": -0.671809,
                "QE03:b1_gradient": 3.22537,
                "QE04:b1_gradient": -3.20496,
}

parameters = {
    "impact_configuration": impact_configuration, 
    "impact_settings": impact_settings, 
    "distgen_configuration": distgen_configuration, 
    "distgen_settings": distgen_settings,
    "archive_filesystem_identifier": "local",
    "dashboard_image_filesystem_identifier": "local",
    "archive_dir": archive_dir,
    "dashboard_dir": dashboard_dir, 
    "pv_collection_isotime": pv_collection_isotime.isoformat()
}
parameters.update(distgen_inputs)
parameters.update(impact_inputs)

In [10]:
flow_run = model.deployment.flow.prefect_flow.run(
    **parameters
)

[2022-10-20 21:25:57-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'lume-distgen-impact-cu-inj'


INFO:prefect.FlowRunner:Beginning Flow run for 'lume-distgen-impact-cu-inj'


[2022-10-20 21:25:57-0700] INFO - prefect.TaskRunner | Task 'L0B_phase:dtheta0_deg': Starting task run...


INFO:prefect.TaskRunner:Task 'L0B_phase:dtheta0_deg': Starting task run...


[2022-10-20 21:25:57-0700] INFO - prefect.TaskRunner | Task 'L0B_phase:dtheta0_deg': Finished task run for task with final state: 'Success'


...

INFO:prefect.TaskRunner:Task 'save_db_result': Starting task run...


[2022-10-20 21:27:59-0700] INFO - prefect.TaskRunner | Task 'save_db_result': Finished task run for task with final state: 'Success'


INFO:prefect.TaskRunner:Task 'save_db_result': Finished task run for task with final state: 'Success'


[2022-10-20 21:27:59-0700] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


INFO:prefect.FlowRunner:Flow run SUCCESS: all reference tasks succeeded


## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory. 
After running the next cell, you'll be able to see the running container in your docker desktop and examine the flow using the Prefect UI at http://localhost:8080/default?flows.

In [12]:
# use mounted filesystem
parameters.update(
    {
        "archive_filesystem_identifier": "mounted",
        "dashboard_image_filesystem_identifier": "mounted",
    }
)
# use mounted filesystem path
parameters["impact_configuration"]["workdir"] = "/lume-services/data"

model.run(
    parameters = parameters
)

# Get results:

In [10]:
results = model.get_results()
results

[Result(project_name='test', id='634dc396801c25354c37e308', flow_id='c53eb190-bbb0-4d9f-ace6-756f7a44a832', inputs={'distgen:r_dist:sigma_xy:value': 0.1376666529, 'distgen:t_dist:length:value': 12.374624528658504, 'distgen:total_charge:value': 4.005441585e-05, 'SOL1:solenoid_field_scale': 0.0874263114, 'CQ01:b1_gradient': 0.0035238095224, 'SQ01:b1_gradient': 0.0035238095224, 'L0A_phase:dtheta0_deg': -8.8997, 'L0A_scale:voltage': 70000000000000.0, 'end_mean_z': 4.6147002}, outputs={'end_n_particle': 5183.83740234375, 'end_mean_gamma': 148.83367919921875, 'end_sigma_gamma': 0.48476168513298035, 'end_mean_x': 1.3811709322908428e-06, 'end_mean_y': 4.753492248710245e-07, 'end_norm_emit_x': 1.7238378859474324e-05, 'end_norm_emit_y': 1.3315763681021053e-05, 'end_norm_emit_z': 0.00019549435819499195, 'end_sigma_x': 0.002542009809985757, 'end_sigma_y': 0.0027486709877848625, 'end_sigma_z': 0.0009321933030150831, 'end_mean_px': -78.1666259765625, 'end_mean_py': -12.492935180664062, 'end_mean_pz'

In [11]:
results_df = model.get_results_df()
results_df

,project_name,flow_id,distgen:r_dist:sigma_xy:value,distgen:t_dist:length:value,distgen:total_charge:value,SOL1:solenoid_field_scale,CQ01:b1_gradient,SQ01:b1_gradient,L0A_phase:dtheta0_deg,L0A_scale:voltage,...,end_cov_y__py,end_cov_z__pz,x:y,out_ymax,out_xmax,out_ymin,out_xmin,date_modified,unique_hash,result_type_string
0,test,c53eb190-bbb0-4d9f-ace6-756f7a44a832,0.137667,12.374625,0.00004,0.087426,0.003524,0.003524,-8.8997,7.000000e+13,...,1197.300171,-13.803169,"[[1.5396431061364324e-15, 4.492893568714936e-1...",0.010092,0.014072,-0.009958,-0.013706,2022-10-17 21:05:14.830,86715642b0fc6fbe33d2b057af71afe4,lume_services.results.generic.Result


## Load model using model id
Once your model has been registered, you can use the `Model` api object to interact with your model without running the above registration steps. Let's load a new model object using the model_id we registered above.

In [13]:
from importlib import reload 
import lume_services
reload(lume_services.models)
from lume_services.models import Model

#model_id = model.metadata.model_id
loaded_model = Model(model_id=1)

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


In [15]:
loaded_model.load_deployment()

Model(                     model_id=1,                     created=datetime.datetime(2022, 10, 17, 21, 4, 22),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu',                     description='lcls-cu-inj-nn'                 )

## Load existing model object
Loading a model using the load_deployment method without passing a deployment_id will load the latest deployment registered for the model.

In [16]:
loaded_model.load_deployment()
loaded_model.deployment

INFO:lume_services.models.model:Loading latest deployment.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 ORDER BY deployment.deploy_date DESC
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow.flow_id, flow.flow_name, flow.project_name, flow.deployment_id 
FROM flow 
WHERE flow.deployment_id = :deployment_id_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT project.project_name, project.description 
FROM project 
WHERE project.project_name = :project_name_1
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT flow_of_flows._id, flow_of_flows.parent_flow_id, flow_of_flows.flow_id, flow_of_flows.position 
FROM flow_of_flows 
WHERE flow_of_fl

Deployment(metadata=Deployment(                 deployment_id=1,                 model_id=1,                 version='0.0.12',                 deploy_date=datetime.datetime(2022, 10, 17, 21, 4, 40)),                 asset_dir=None,                 source='https://github.com/jacquelinegarrahan/lume-lcls-cu-inj-nn/releases/download/v0.0.12/lume_lcls_cu_inj_nn-0.0.12.tar.gz',                 sha256='7d7827f8c871e0cfa266343c2aed990a6c5909935d72e87674e013b4d6bb4fae',                 image='scr.svc.stanford.edu/jgarra/lume-lcls-cu-inj-nn:v0.0.12',                 is_live=True                 package_import_name='lume_lcls_cu_inj_nn'                 ), project=Project(metadata=Project(                 project_name='test',                 description='my_description',                 )), flow=Flow(name='lume-lcls-cu-inj-nn', flow_id='c53eb190-bbb0-4d9f-ace6-756f7a44a832', project_name='test', prefect_flow=None, parameters=None, mapped_parameters=None, task_slugs=None, labels=['lume-services'],

In [17]:
result = loaded_model.run_and_return(
    parameters={
                        "distgen:r_dist:sigma_xy:value": 0.4130, 
                        "distgen:total_charge:value": 250.0, 
                        "distgen:t_dist:length:value":7.499772441611215, 
                        "SOL1:solenoid_field_scale": 0.17, 
                        "CQ01:b1_gradient":-0.0074,
                        "SQ01:b1_gradient": -0.0074,
                        "L0A_phase:dtheta0_deg": -8.8997,
                        "L0A_scale:voltage": 70000000.0,
                        "distgen:t_dist:length:value": 7.499772441611215,
                        "end_mean_z": 4.6147002
    },
    task_name="save_db_result" # Want to get the result from the save_db_result task
)
result

INFO:lume_services.services.scheduling.backends.server:Creating Prefect flow run for fb70c882-183e-41e0-8f15-d72c4f32816a with parameters {'distgen:r_dist:sigma_xy:value': 0.413, 'distgen:total_charge:value': 250.0, 'distgen:t_dist:length:value': 7.499772441611215, 'SOL1:solenoid_field_scale': 0.17, 'CQ01:b1_gradient': -0.0074, 'SQ01:b1_gradient': -0.0074, 'L0A_phase:dtheta0_deg': -8.8997, 'L0A_scale:voltage': 70000000.0, 'end_mean_z': 4.6147002} and run_config {"labels": ["lume-services"], "env": {"PREFECT__CONTEXT__PROJECT_NAME": "test"}, "image": "scr.svc.stanford.edu/jgarra/lume-lcls-cu-inj-nn:v0.0.12", "host_config": null, "ports": null}
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 10, 14, 22, 56, 38, 379453, tzinfo=Timezone('+00:00')), level=20, message='Entered state <Scheduled>: Flow run scheduled.')
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 10, 14, 22, 56, 42, 707908, tzinfo=Timezone

INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 10, 14, 22, 56, 58, 423860, tzinfo=Timezone('+00:00')), level=20, message="Task 'prepare_lume_model_variables': Starting task run...")
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 10, 14, 22, 56, 58, 523731, tzinfo=Timezone('+00:00')), level=20, message="Task 'prepare_lume_model_variables': Finished task run for task with final state: 'Success'")
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 10, 14, 22, 56, 58, 586398, tzinfo=Timezone('+00:00')), level=20, message="Task 'preprocessing_task': Starting task run...")
INFO:lume_services.services.scheduling.backends.server:FlowRunLog(timestamp=DateTime(2022, 10, 14, 22, 56, 58, 739239, tzinfo=Timezone('+00:00')), level=20, message="Task 'preprocessing_task': Finished task run for task with final state: 'Success'")
INFO:lume_services.services.scheduling.backends.

Result(project_name='test', id='6349e8f18a6985b236762622', flow_id='fb70c882-183e-41e0-8f15-d72c4f32816a', inputs={'distgen:r_dist:sigma_xy:value': 0.1376666529, 'distgen:t_dist:length:value': 12.374624528658504, 'distgen:total_charge:value': 4.005441585e-05, 'SOL1:solenoid_field_scale': 0.0874263114, 'CQ01:b1_gradient': 0.0035238095224, 'SQ01:b1_gradient': 0.0035238095224, 'L0A_phase:dtheta0_deg': -8.8997, 'L0A_scale:voltage': 70000000000000.0, 'end_mean_z': 4.6147002}, outputs={'end_n_particle': 5183.83740234375, 'end_mean_gamma': 148.83367919921875, 'end_sigma_gamma': 0.48476168513298035, 'end_mean_x': 1.3811709322908428e-06, 'end_mean_y': 4.753492248710245e-07, 'end_norm_emit_x': 1.7238378859474324e-05, 'end_norm_emit_y': 1.3315763681021053e-05, 'end_norm_emit_z': 0.00019549435819499195, 'end_sigma_x': 0.002542009809985757, 'end_sigma_y': 0.0027486709877848625, 'end_sigma_z': 0.0009321933030150831, 'end_mean_px': -78.1666259765625, 'end_mean_py': -12.492935180664062, 'end_mean_pz':

In [22]:
results_df = loaded_model.get_results_df()
results_df

,project_name,flow_id,distgen:r_dist:sigma_xy:value,distgen:t_dist:length:value,distgen:total_charge:value,SOL1:solenoid_field_scale,CQ01:b1_gradient,SQ01:b1_gradient,L0A_phase:dtheta0_deg,L0A_scale:voltage,...,end_cov_y__py,end_cov_z__pz,x:y,out_ymax,out_xmax,out_ymin,out_xmin,date_modified,unique_hash,result_type_string
0,test,fb70c882-183e-41e0-8f15-d72c4f32816a,0.137667,12.374625,0.00004,0.087426,0.003524,0.003524,-8.8997,7.000000e+13,...,1197.300171,-13.803169,"[[1.5396431061364324e-15, 4.492893568714936e-1...",0.010092,0.014072,-0.009958,-0.013706,2022-10-14 22:55:33.822,84f85f609f96bd0383431a0533b9682d,lume_services.results.generic.Result
1,test,fb70c882-183e-41e0-8f15-d72c4f32816a,0.137667,12.374625,0.00004,0.087426,0.003524,0.003524,-8.8997,7.000000e+13,...,1197.300171,-13.803169,"[[1.5396431061364324e-15, 4.492893568714936e-1...",0.010092,0.014072,-0.009958,-0.013706,2022-10-14 22:56:50.477,84f85f609f96bd0383431a0533b9682d,lume_services.results.generic.Result
